## Step 1: configure AWS account

First of all, your aws account needs to be set up in the following way.

1) Go to [AWS](https://aws.amazon.com/) and *Sign in to console*

2) Click on your username in the upper right corner and select *Security credentials*. Choose *Continue to security credentials*

3) Click on *Access keys* and *Create New Access Keys*. A .csv file containing the **ACCESS KEY ID** and **ACCESS SECRET KEY** will be downloaded. These keys are needed to use AWS remotely. I would reccomend to put the csv file somewhere safe. **This only needs to be done once**.

4) On the right panel (*Details*) click on *Users*. *Create new user* for yourself. Give the user (you ;-) ) a name and *Create*.

5) On the right panel (*Details*) click on *Groups*. *Create new group*. Give it a name, then select *AdministratorAccess* as an attached security policy. Review and create group.

6) When the group has been created, check the box next to it and click *Group Actions* and then *Add User to Group*. Add yourself to the security group. 

4) On the upper right corner click on the name of the region (probably Frankfurt). **Change it to EU (Ireland)**. This is critical because the Frankfurt EC2 compute node cannot be used by Spark to launch a cluster. **Do this before the next step!**

5) Go to *Services* (upper left corner) and select *EC2* in the *Compute* category.

6) In the dashboard on the left click on *Key pairs* under the *Network and security* category. Then click on *Create key pair*. Give it a name and click *Create*. A .pem file will be downloaded to your computer. This is the SSH public-private key pair needed to access any EC2 instances. I suggest to put the file in the `~/.ssh` directory of your computer (for Windows users anywhere works as long as you remember where it is). **You only need one key pair. Each key pair is associated with a region so if you change it back to Frankfurt you need to generate a new one and trash the old one.**

**That's it for setting up your AWS account Yepeeeee :-P!**

## Step 2: create an AWS S3 storage node

1) Go to *Services* and then choose *S3*.

2) Click on *Create Bucket* and choose a region (ideally the same as your EC2 settings). 

That's basically it! You can then click on your storage bucket and upload files. 

## Step 3: launch a n-nodes EC2 cluster running Spark

Now that your AWS account is setup we can use the dedicated tools provided in the Spark distribution to launch a cluster on AWS EC2.

0) [Download](http://spark.apache.org/downloads.html) the spark distribution on your computer. `tar -xvzf` the folder somewhere on your computer. `cd` in the folder. Finally, `cd ./ec2`

1) First, export your AWS credentials found in the `credentials.csv` file: 

    export AWS_SECRET_ACCESS_KEY=yoursecretaccesskey
    export AWS_ACCESS_KEY_ID=youraccesskeyid

1) This is the command to launch a cluster on AWS EC2: `./spark-ec2 -k mykeypair -i ~/.ssh/mykeypair.pem -s 3 --instance-type=t2.medium --region=eu-west-1 --zone=eu-west-1a --copy-aws-credentials launch mycluster`

* `-k` is your keypair name
* `-i` is the path to your keypair
* `-s` the number of nodes you want including the master node
* `--instance-type` is the type of instances you want. I recommend t2.medium instances since they are free
* `--region` and `--zone` are needed for this to work. You can find the different zones available in your region on the EC2 dashboard. 
* `--copy-aws-credentials` is needed to access S3 storage nodes from your cluster
* `mycluster` is the name you want to give to your cluster. 

## ** Important: Do not interrupt the script that will be launched...especially during the `Waiting for SSH-ready state...` part. Spark needs to wait for AWS to setup the instances before installing Spark on them and setting the SSH configs. The set-up of the cluster can take 10-15 minutes. ** 


# Step 4: Verifications and login into the master node

1) Run `./spark-ec2 -k mykeypair -i ~/.ssh/mykeypair.pem --region=eu-west-1 login mycluster` to automatically login to your cluster's master node.

2) Go on the AWS website EC2 service. You should be able to see all the launched instances (master and slaves). You can also find the public DNS of the instances. Access `http://public_dns_of_your_master_node.compute.amazonaws.com:8080/` from your browser. You should be able to see the Spark UI for your cluster with master and slave nodes. You can access individual nodes by SSH'ing into them: `ssh -k your_key_pair.pem root@public_dns_of_your_master_node.compute.amazonaws.com:22`

3) Other useful commands: 

* Pause the cluster (only cost is the RAM needed to retain the data): `./spark-ec2 -k mykeypair -i ~/.ssh/mykeypair.pem --region=eu-west-1 stop mycluster`
* Restart the cluster: `./spark-ec2 -k mykeypair -i ~/.ssh/mykeypair.pem --region=eu-west-1 start mycluster`
* Destroy the cluster. **This wipes everything in memory!**: `./spark-ec2 -k mykeypair -i ~/.ssh/mykeypair.pem --region=eu-west-1 destroy mycluster`

# Step 5: Reading/writing to S3 bucket from spark cluster

Assuming you launched the cluster with the `--copy-aws-credentials` flag, you should be able to access your S3 bucket in the following way: 

    val x = sc.textFile("s3n://my_bucket_adress/data/*")
    x.take(5)
    x.saveAsTextFile("s3n://my_bucket_adress/data2/") # this writes to another file in the same bucket 

# Step 6: Using Jupyter notebook remotely the cluster

1) Start you cluster:

        ./spark-ec2 -k bigdive5-eu-west -i ~/bigdive5-eu-west.pem --region=eu-west-1 start mycluster
   
2) Login to the master node: 

        ./spark-ec2 -k bigdive5-eu-west -i ~/bigdive5-eu-west.pem --region=eu-west-1 login mycluster
    
3) Before we need to update the python distribution to 2.7 and pip to install jupyter notebook on the master node:

        # install python 2.7 and change default python symlink 
        sudo yum install python27-devel -y 
        sudo rm /usr/bin/python
        sudo ln -s /usr/bin/python2.7 /usr/bin/python 

        # yum still needs 2.6, so write it in and backup script 
        sudo cp /usr/bin/yum /usr/bin/_yum_before_27 
        sudo sed -i s/python/python2.6/g /usr/bin/yum 
        sudo sed -i s/python2.6/python2.6/g /usr/bin/yum 

        # should display now 2.7.5 or later: 
        python -V 

        # now install pip for 2.7 
        sudo curl -o /tmp/ez_setup.py https://bootstrap.pypa.io/ez_setup.py


        sudo /usr/bin/python27 /tmp/ez_setup.py 
        sudo /usr/bin/easy_install-2.7 pip 
        
        # should display current versions:
        pip -V

4) Now install jupyter notebook with pip:

        pip install jupyter notebook

5) Launch IPython and create an encrypted password to login to the notebook:

        ipython
        In [1]:from IPython.lib import passwd
        In [2]:passwd()

Enter a password and verify it. **Note down the sha1 hash somewhere!**

6) Create a config file for the jupyter notebook server:

    jupyter notebook --generate-config
    
7) Create a SSL certificate to access the notebook through a secure HTTPS connexion with the password created earlier:

        mkdir certs
        cd certs
        sudo openssl req -x509 -nodes -days 365 -newkey rsa:1024 -keyout mycert.pem -out mycert.pem
        
Answer all the questions that the openssl script will ask you.

8) Modify the config file of the notebook server:

        cd ~/.jupyter
        
9) Add the following lines at the begginning of the jupyter_notebook_config.py file. ** Replace the `c.NotebookApp.password` field with the `sha1` hash you generated before **:
    
        c = get_config()

        # Kernel config
        c.IPKernelApp.pylab = 'inline'  # if you want plotting support always in your notebook

        # Notebook config
        c.NotebookApp.certfile = u'/root/certs/mycert.pem' #location of your certificate file
        c.NotebookApp.ip = '*'
        c.NotebookApp.open_browser = False  #so that the ipython notebook does not opens up a browser by default
        c.NotebookApp.password = u'sha1:68c136a5b064...'  #the encrypted password we generated above
        # It is a good idea to put it on a known, fixed port
        c.NotebookApp.port = 8888
        
10) Now it's time to update you security group to allow https connexion on port 8888 on your nodes. Go to your AWS EC2 dashbord and click on _Security Groups_ in the _Network and Security_ category. Find the security group of your master node (probably the first one) and click on it. Click on the inbound tab and then click on Edit. Add a custom TCP Rule on _Port Range_ 8888 with _Source_ set as _Anywhere_ and _0.0.0.0/0_. Save the inbound rules. 

11) Now go back to the terminal connected to your master node. Start the jupyter notebook:

        IPYTHON_OPTS="notebook" /root/spark/bin/pyspark
        
12) **That's it**! You can now access your notebook in your browser by going to `https://ec2.your.master.node.dns:8888`. If your browser tells you it's not safe click on advanced (in Chrome) and `access the adress anyway`

13) To check that you are running the spark API just type the following in the notebook:

        sc

## That's pretty much it! I will continue to update the notebook as we go